# Bagging (Bootsrap aggregating)
- bagging algorithm with for loop
- predict price

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
os.getcwd()

'/Users/chy/python programming'

In [4]:
#load data
data = pd.read_csv("./Data/kc_house_data.csv")
data.head()

,id,date,price,bedrooms,bathrooms,floors,waterfront,condition,grade,yr_built,yr_renovated,zipcode,lat,long
0,7129300520,20141013T000000,221900.0,3,1.00,1.0,0,3,7,1955,0,98178,47.5112,-122.257
1,6414100192,20141209T000000,538000.0,3,2.25,2.0,0,3,7,1951,1991,98125,47.7210,-122.319
2,5631500400,20150225T000000,180000.0,2,1.00,1.0,0,3,6,1933,0,98028,47.7379,-122.233
3,2487200875,20141209T000000,604000.0,4,3.00,1.0,0,5,7,1965,0,98136,47.5208,-122.393
4,1954400510,20150218T000000,510000.0,3,2.00,1.0,0,3,8,1987,0,98074,47.6168,-122.045


In [5]:
nCar = data.shape[0] #num of data
nVar = data.shape[1] #num of variable
print('num of data: %d' %nCar, 'num of variable: %d' %nVar)

num of data: 21613 num of variable: 14


In [6]:
#remove data which is no meaning
data = data.drop(['id', 'date', 'zipcode', 'lat', 'long'], axis = 1)

In [7]:
#split train & test
feature_columns = list(data.columns.difference(['price'])) #except price
X =data[feature_columns]
y = data['price']
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size = 0.3, random_state = 42) #train:test = 7:3
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape) #check data

(15129, 8) (6484, 8) (15129,) (6484,)


In [9]:
# verify with linear regression
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

sm_train_x = sm.add_constant(train_x, has_constant = 'add') #add bias
sm_model = sm.OLS(train_y, sm_train_x) #build a model
fitted_sm_model = sm_model.fit() #training
fitted_sm_model.summary() #check structure of train model
#check coef (positive means direct proportion with price)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.595
Model:                            OLS   Adj. R-squared:                  0.595
Method:                 Least Squares   F-statistic:                     2776.
Date:                Tue, 03 Aug 2021   Prob (F-statistic):               0.00
Time:                        16:38:16   Log-Likelihood:            -2.0826e+05
No. Observations:               15129   AIC:                         4.165e+05
Df Residuals:                   15120   BIC:                         4.166e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const         7.186e+06   1.73e+05     41.548      0.000    6.85e+06    7.52e+06
bathrooms     1.303e+05   3960.833     32.889      0.000    1.23e+05    1.38e+05
bedrooms     -2224.7910   2382.356     -0.934      0.350   -6894.497    2444.915
condition     1.641e+04   3169.013      5.178      0.000    1.02e+04    2.26e+04
floors        1946.3052   4336.838      0.449      0.654   -6554.422    1.04e+04
grade         1.956e+05   2199.540     88.924      0.000    1.91e+05       2e+05
waterfront    7.555e+05   2.26e+04     33.479      0.000    7.11e+05       8e+05
yr_built     -4300.7865     88.073    -48.832      0.000   -4473.420   -4128.153
yr_renovated    12.7325      5.043      2.525      0.012       2.847      22.618
==============================================================================
Omnibus:                    13447.374   Durbin-Watson:                   1.994
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1684794.827
Skew:                           3.763   Prob(JB):                         0.00
Kurtosis:                      54.147   Cond. No.                     1.82e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.82e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [10]:
#predict
sm_test_x = sm.add_constant(test_x, has_constant = 'add') # add bias to test data
sm_model_predict = fitted_sm_model.predict(sm_test_x) # predict test data
print("RMSE: {}".format(sqrt(mean_squared_error(sm_model_predict, test_y)))) # RMSE
print(fitted_sm_model.params) 

RMSE: 239804.29670858165
const           7.185671e+06
bathrooms       1.302689e+05
bedrooms       -2.224791e+03
condition       1.641020e+04
floors          1.946305e+03
grade           1.955909e+05
waterfront      7.555423e+05
yr_built       -4.300787e+03
yr_renovated    1.273246e+01
dtype: float64


# Comparing with Bagging (with for loop)

In [18]:
import random
bagging_predict_result = []
#execute 10 times -> increase times to 30
for _ in range(30):
    data_index = [data_index for data_index in range(train_x.shape[0])] #num of data 
    random_data_index = np.random.choice(data_index, train_x.shape[0]) #Restoration Extraction 
    print(len(set(random_data_index)))
    sm_train_x = train_x.iloc[random_data_index, ] 
    sm_train_y = train_y.iloc[random_data_index, ] 
    sm_train_x = sm.add_constant(sm_train_x, has_constant = 'add') 
    sm_model = sm.OLS(sm_train_y, sm_train_x) 
    fitted_sm_model = sm_model.fit() 
    
    sm_test_x = sm.add_constant(test_x, has_constant = 'add') 
    sm_model_predict = fitted_sm_model.predict(sm_test_x) 
    bagging_predict_result.append(sm_model_predict) # add resultto list    

9581
9593
9509
9659
9573
9575
9570
9554
9583
9561
9522
9593
9599
9562
9562
9531
9618
9534
9581
9564
9586
9582
9627
9522
9586
9567
9572
9608
9583
9641


In [19]:
# result (1st predict result, can check 0 - 9)
bagging_predict_result[0]

735      5.509041e+05
2830     7.169969e+05
4106     1.098260e+06
16218    1.471676e+06
19964    6.934913e+05
             ...     
12606    5.981546e+05
14393    6.819178e+05
6899     3.161406e+05
85       8.845763e+05
21363    4.257304e+05
Length: 6484, dtype: float64

In [20]:
#calculate mean with bagging result
bagging_predict = []
for lst2_index in range(test_x.shape[0]): #index of data
    temp_predict = []
    for lst_index in range(len(bagging_predict_result)): #index of list of predict result
        temp_predict.append(bagging_predict_result[lst_index].values[lst2_index])
    bagging_predict.append(np.mean(temp_predict))

In [21]:
bagging_predict
print("RMSE: {}".format(sqrt(mean_squared_error(bagging_predict, test_y))))

RMSE: 239844.56485097055


# Bagging2
- with bagging package

In [22]:
from sklearn.linear_model import LinearRegression
regression_model = LinearRegression() 
linear_model1 = regression_model.fit(train_x, train_y) 
predict1 = linear_model1.predict(test_x)
print("RMSE: {}".format(sqrt(mean_squared_error(predict1, test_y)))) # RMSE 

RMSE: 239804.2967085815


In [23]:
from sklearn.ensemble import BaggingRegressor
bagging_model = BaggingRegressor(base_estimator = regression_model, 
                                 n_estimators = 5, # sampling 5 times
                                 verbose = 1) 
linear_model2 = bagging_model.fit(train_x, train_y) 
predict2 = linear_model2.predict(test_x) 
print("RMSE: {}".format(sqrt(mean_squared_error(predict2, test_y)))) 

RMSE: 239701.00837491546


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


In [25]:
#increase num of sampling
from sklearn.ensemble import BaggingRegressor
bagging_model = BaggingRegressor(base_estimator = regression_model, 
                                 n_estimators = 30, # sampling 30 times -> same with increasing index of for loop before
                                 verbose = 1) 
linear_model2 = bagging_model.fit(train_x, train_y) 
predict2 = linear_model2.predict(test_x) 
print("RMSE: {}".format(sqrt(mean_squared_error(predict2, test_y)))) 

RMSE: 239734.2919085348


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
